# Homework 3: EDA of Bike Sharing

## Introduction

This homework has four goals: 

* to learn how to read plain text delimited data into pandas
* to gain experience preparing data for analysis
* to use EDA to learn about your data 
* to learn how to make informative plots

This assignment includes both specific tasks to perform and open-ended questions to investigate. The open-ended questions ask you to think creatively and critically in how you plot the data and summarize information gleaned from these visualizations.

There are 4 parts to this assignment: 
- data preparation
- exploring the distribution of riders
- exploring the relationship between time and rider counts
- and exploring the relationship between weather and rider counts. 

The data preparation section gives you specific tasks to perform. Then in each of the next three sections, you are asked to create 2 plots. The first plot is described in some detail and the second plot is one of your choosing.

Be sure to choose a plot that provides additional information about the distributions and relationships in the data. Remember that we are interested in how ride sharing works.  Therefore, plots that, for example examine the weather in DC *without reference to ride sharing are not interesting*.

**Be sure that your plots have informative titles, axis labels, and legends.**

In [ ]:
# For instructor use only. Call this function to force refresh okpy tests
def refresh():
    import sys
    keys = [k for k in sys.modules.keys() if 'ok_tests' in k]
    for k in keys:
        del sys.modules[k]
    global ok
    ok = Notebook('hw3.ok')

In [ ]:
# Run this cell to set up your notebook

import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

from IPython.display import display, Latex, Markdown
from client.api.notebook import Notebook
ok = Notebook('hw3.ok')


In [ ]:
ok.auth(force=True) # Change False to True if you are getting errors authenticating

In [ ]:
## This can be used by TAs to fix students directories if needed.
# # Log into OkPy
# import os
# auth_refresh = os.path.join(os.path.expanduser('~'), '.config', 'ok', 'auth_refresh')
# if os.path.exists(auth_refresh):
#     os.remove(auth_refresh)

## Loading Bike Sharing Data
The data we are exploring is data on bike sharing in Washington D.C and can be found in `data/bikeshare.txt`.

The variables in this data frame are defined as:
```
instant: record index
dteday : date
season : season (1:spring, 2:summer, 3:fall, 4:winter)
yr : year (0: 2011, 1:2012)
mnth : month ( 1 to 12)
hr : hour (0 to 23)
holiday : whether day is holiday or not
weekday : day of the week
workingday : if day is neither weekend nor holiday
weathersit :
    1: Clear or partly cloudy
    2: Mist + clouds
    3: Light Snow or Rain
    4: Heavy Rain or Snow
temp : Normalized temperature in Celsius (divided by 41)
atemp: Normalized feeling temperature in Celsius (divided by 50)
hum: Normalized percent humidity (divided by 100)
windspeed: Normalized wind speed (divided by 67)
casual: count of casual users
registered: count of registered users
cnt: count of total rental bikes including casual and registered
```

In the following we load the raw data into the dataframe `bike`. 

In [ ]:
bike = pd.read_csv("data/bikeshare.txt")
bike.head()

## Data Preparation
Many of the variables that are numeric/integer should be factors. These include holiday, weekday, workingday, and weathersit. In the following question we convert these 4 variables to factors and use appropriate labels for the levels. In particular use the 3 letter label, e.g., Sun, Mon, … for weekday. You may simply use yes/no for holiday and workingday. 

In this exercise we will **overwrite the corresponding variables in the data frame.** However be sure to write a comment in your code about the transformation and leave the underlying datafile `data/bikeshare.txt` unmodified.

### Question 1

#### Question 1a (Decoding `holiday`)
Figure out whether a 0 means yes or no for the holiday column of each row (there are fewer holidays than other days). Translate the numeric entry into a `'yes'` or `'no'` and set `num_holidays` equal to the number of entries in the table corresponding to a holiday. 

In [ ]:
#SOLUTION CELL
bike['holiday'].replace({0:'no', 1:'yes'}, inplace=True)
num_holidays = bike['holiday'].value_counts()['yes']
num_holidays

In [ ]:
_ = ok.grade('q01a')
_ = ok.backup()

#### Question 1b (Decoding `weekday`, `workingday`, and `weathersit`)


Finish decoding the `weekday`, `workingday`, and `weathersit` fields:

1. **`'weekday'`:** It turns out that Monday is the day with the most holidays.  Use that information to change the `'weekday'` column to use the 3 letter label (`'Sun'`, `'Mon'`, ...) instead of its current numerical values.
1. **`'workingday'`:** Convert to `'yes'` and `'no'`.
1. **`'weathersit'`:** You should replace each value with one of `'Clear'`, `'Mist'`, `'Light'`, or `'Heavy'`.

In [ ]:
bike['weekday'].replace({0:'Sun', 1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat'}, inplace=True)
bike['workingday'].replace({0:'no', 1:'yes'}, inplace=True)
bike['weathersit'].replace({1:'Clear', 2:'Mist', 3:'Light', 4:'Heavy'}, inplace=True)
bike.head()

In [ ]:
_ = ok.grade('q01b')
_ = ok.backup()

#### Question 1c (Computing Daily Total Counts)

The granularity of this data is at the hourly level.  However, for some of the analysis we will also want to compute daily statistics.  In particular, in the next few questions we will be analyzing the number of daily registered and unregistered users.

Construct a table (dataframe) consisting of:
* `'casual'`: total casual riders for each day
* `'registered'`: total registered riders for each day
* `'workingday'`: whether that day is a working day or not (yes or no)

You will want to use the `groupby` and `aggregate` functions as well as the `sum` and `last` aggregation operators. 

In [ ]:
daily_counts = bike.groupby(['dteday']).aggregate({"casual": sum, "registered": sum, "workingday": "last"})
daily_counts.head()

In [ ]:
_ = ok.grade('q01c')
_ = ok.backup()

## Exploring the Distribution of Riders

Let's begin by comparing the distribution of the daily counts of casual and registered riders.  

### Question 2
#### Question 2a

Use the `sns.distplot` function to create a plot that overlays the distribution of the daily counts of casual and registered users. The temporal granularity of the records should be daily counts, which you should have after completing question 1c.

In [ ]:
sns.distplot(daily_counts['casual'], label='casual')
sns.distplot(daily_counts['registered'],  label='registered', color='green')
plt.legend()
plt.xlabel("Rider Count")

#### Question 2b

Describe the differences you notice between the density curves for casual and registered riders. Be sure to consider features such as modes, symmetry and skewness, tails, gaps and unusual values. Also comment on the spread of the distributions.

In [ ]:
q2b_answer = r"""


The casual riders distribution has a sharp peak at 1000 that may be bimodal. 
This distribution is skewed right and has a long right tail with a small set of daily counts around 2500.

The distribution of registered riders has a more symmetric distribution centered around almost 4000 daily riders.
This distribution does not have heavy skew. It's spread is much wider than the casual riders. 

"""

display(Markdown(q2b_answer))

### Question 2c

The density plots do not show us how the daily counts for registered and casual riders vary together. Use `sns.lmplot` to make a scatter plot to investigate the relationship between casual and registered counts. Color the points in the scatterplot according to whether or not the day is working day. There are many points in the scatter plot so make them small  to help with over plotting.

In [ ]:
sns.set(font_scale=1.5)
sns.lmplot(x="casual", y="registered", hue="workingday",
           data=bike, fit_reg=False, size=8, scatter_kws={"s": 10})

### Question 2d

What does this scatterplot seem to reveal about the relationship (if any) between casual and registered riders and whether or not the day is on the weekend?
Why might we be concerned with overplotting in examining this relationship? 


In [ ]:
q2d_answer = r"""

There appears to be a linear relationship between the counts for registered and casual riders,
and this relationship depends on whether the day is a work day or a weekend day.

Given the overplotting, it is difficult to discern the relationships because
for example, we can't see if there might be blue points that have been 
plotted over by green points and vice versa.


"""

display(Markdown(q2d_answer))

## Question 3

### Question 3a Bivariate Kernel Density Plot
 
The scatter plot you made in question 2c makes clear the separation between the work days and weekend days.  However, the overplotting
makes it difficult to see the density of the joint counts. To address this
issue, let's try visualizing the data with another technique, the kernel density plot.

You will want to read up on the documentation for `sns.kdeplot` which can be found at https://seaborn.pydata.org/generated/seaborn.kdeplot.html

The result we wish to achieve should be a plot that looks like this:

<img src='bivariate_kde_of_daily_rider_types.png' >

Making this plot can be complicated so we will provide a walkthrough below, feel free to use whatever method you wish however if you do not want to follow the walkthrough.

In [ ]:
ind = daily_counts['workingday'] == "yes"

ax = sns.kdeplot(daily_counts.loc[ind, "casual"], daily_counts.loc[ind, "registered"], 
            cmap="Reds",  shade_lowest=False, label="weekday")
ax = sns.kdeplot(daily_counts.loc[~ind, "casual"], daily_counts.loc[~ind, "registered"],
            cmap="Blues", shade_lowest=False)

plt.savefig("bivariate_kde_of_daily_rider_types.png")

#### Question 3b

What does the contour plot reveal about the relationship between casual and registered riders for both weekdays and weekends? How is it an improvement over the scatter plot you created in Q2c?

In [ ]:
q3b_answer = r"""

On the weekends, the casual and registered riders have a roughly linear relationship. 
During the work week, the relationship also appears roughly linear,
but the slope is much higher. 

Here we can see the shape of the relationship because high density regions 
are revealed that we were not able to see due to overplotting in the scatter plot.

"""

display(Markdown(q3b_answer))

#### Question 3c

As an alternative approach, construct the following set of three plots where the main plot shows the contours of daily counts for registered and casual riders and the two "margin" plots provide the univariate distribution of each of these variables. 

<img src="joint_distribution_of_daily_rider_types.png"/>

You might find `sns.jointplot` helpful. You will want to explore the `kind` parameter in `jointplot` to achieve the plot above. Also, consider using `set_axis_labels` to rename your axes.

In [ ]:
sns.jointplot(x="casual", y="registered", data=daily_counts,  kind="kde")\
   .set_axis_labels('Daily Count Casual Riders', 'Daily Count Registered Riders')

plt.savefig("joint_distribution_of_daily_rider_types.png")

## Exploring Ride Sharing and Time

### Question 4a

Plot number of riders for each hour of each day in the month of June in 2011. 

Make sure to add descriptive x-axis and y-axis labels and create a legend to distinguish the line for casual riders and the line for registered riders. The end result should look like this:

<img src="june_riders.png" width='800px'/>

In [ ]:
june = bike[(bike["mnth"]==6) & (bike["yr"]==0)].copy()
june['day'] = (june['instant'] - june['instant'].min())/24.0
june = june.set_index("day")

# plt.figure(figsize=(10, 7))
_ = june[["casual", "registered"]].plot()
_ = plt.legend(loc=2)
_ = plt.ylabel("Number of Riders")
_ = plt.xlabel("Day of the Month")

## Uncomment this to save new plot for question.
plt.savefig("june_riders.png")

#### Question 4b

This plot has several interesting features. How do the number of casual and registered riders compare for different days of the month? What is an interesting trend and pattern you notice between the lines?

In [ ]:
q4b_answer = r"""

We can see several things in this line plot. We see daily patterns in rental. 
With the weekends having far fewer registered users and more casual riders. 
The number of casual riders is always less than the number of registerred riders, 
but on the week end, the counts are much closer together. 

"""

display(Markdown(q4b_answer))

## Question 5 (Understanding Daily Patterns)

Let's examine the behavior or riders by plotting the average number of riders for each hour of the day over the entire dataset stratified by rider type.  

Your plot should look like the following:

<img src="diurnal_bikes.png" width="800px"/>

In [ ]:
# plt.figure(figsize=(10, 7))
data = bike.groupby("hr")[["casual","registered"]].mean()
_ = data[["casual", "registered"]].plot()
_ = plt.legend(loc=2)
_ = plt.xlabel("Hour of the Day")
_ = plt.ylabel("Average Count")

## Uncomment this to render the instruction picture
plt.savefig("diurnal_bikes.png")

#### Question 5b

What can you observe from the plot, what does those peaks mean?

In [ ]:
q5b_answer = r"""

In the above plot we see strong evidence of daily patterns in both datasets.  
The casual riders appear to ride throughout the day with peak hours in the mid-afternoon. 
Alternatively, while the registered riders also ride more during the day than at night there are
very strong spikes during the morning and evening commute hours with a small bump during lunch.  

"""

display(Markdown(q5b_answer))

## Exploring Ride Sharing and Weather
### Question 6
Now let's examine how the weather is affecting rider's behavior. First let's look at how the proportion of casual riders changes as weather changes.

#### Question 6a
Create a new column `propCasual` in the `bike` dataframe representing the proportion of casual riders comparing to the total number of riders.



In [ ]:
bike["propCasual"] = bike["casual"]/bike["cnt"]

In [ ]:
_ = ok.grade('q06a')
_ = ok.backup()

#### Question 6b
In order to examine the relationship between proportion of casual riders and temerature, we can make some bivariate scatterplot using `sns.swarmplot` or `sns.lmplot`. We can even use color/hue to encode the information about day of week. Run the following cells to create such plots. (Plot contains lots of points, so it may a while for it to render)


In [ ]:
sns.lmplot(data=bike,x="temp",y="propCasual",hue="weekday",scatter_kws={"s": 20},size=10)

As you can see from the scatterplot, many points are overlapping. Though the plot can show some trends, it would be hard to verify your findings. Therefore we could fit some curves to summarize the data and plot the curves instead.

Basically you will need to make a plot like this: Fit and plot curves using data from different weekdays. 

<img src="curveplot_temp_propCasual.png" width=500px>

You will need to use the function [`statsmodels.nonparametric.smoothers_lowess.lowess`](http://www.statsmodels.org/dev/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html) to fit a curve from two sequences `x` and `y`. You may find the example below helpful.



In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
# Make noisy data
xobs = np.sort(np.random.rand(100)*4.0 -2)
yobs = np.exp(xobs) + np.random.randn(100)/2.0
plt.plot(xobs, yobs, 'o', label = "Raw Data")

# Predict 'smoothed' valued for observations
ysmooth = lowess(yobs, xobs, return_sorted=False)
plt.plot(xobs, ysmooth, 'r-', label ="Smoothed Estimator")
plt.legend()

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
days = ['Sun','Mon', 'Tues', 'Wed', 'Thu', 'Fri', 'Sat']
plt.figure(figsize=(8,8))
for day in days:
    z=lowess(bike[(bike["weekday"]==day)]["propCasual"],bike[bike["weekday"]==day]["temp"],return_sorted=False)    
    plt.plot(bike[bike["weekday"]==day]["temp"],z,label = day)
plt.xlabel("Temperature")
plt.ylabel("propCasual")
plt.legend()
#plt.savefig("curveplot_temp_propCasual")

#### Question 6c
What can you discover from the curve plot? How is the propCasual changing?

In [ ]:
#SOLUTION CELL
q6c_answer = r"""
1. As temperature increase, the proportion of casual riders increase as well.
2. Weekends (Saturday, Sunday) have higher proportion of casual riders
"""

display(Markdown(q6c_answer))

In [ ]:
list(bike['weekday'].iloc[::2000])

## Submission

Run the cell below to run all the OkPy tests at once:

In [ ]:
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("ok_tests") if q.startswith('q')]

Now, run the cell below to submit your assignment to OkPy. The autograder should email you shortly with your autograded score. The autograder will only run once every 30 minutes.

**If you're failing tests on the autograder but pass them locally**, you should simulate the autograder by doing the following:

1. In the top menu, click Kernel -> Restart and Run all.
2. Run the cell above to run each OkPy test.

**You must make sure that you pass all the tests when running steps 1 and 2 in order.** If you are still failing autograder tests, you should double check your results.

In [ ]:
_ = ok.submit()